In [1]:
# 데이터 분석을 위한 패키지
import numpy as np
import pandas as pd

In [2]:
# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [3]:
import FinanceDataReader as fdr

In [4]:
import koreanize_matplotlib

In [5]:
import datetime
from dateutil.relativedelta import relativedelta

In [6]:
from data_loader import load_sector_data

In [7]:
# 모든 컬럼,행을 출력하도록 설정
pd.set_option('display.max_columns', None) # None으로 설정하면 모든 컬럼 출력
pd.set_option('display.max_rows', None) #None으로 설정하면 모든 행 출력

In [10]:
# 회귀선의 의미 : 동일한 평균수익률을 가진 다른 업종/종목에 비해 변동성이 적어 리스크가 낮다는 인사이트를 제공할 수 있음
# plotly는 lmplot을 직접적으로 제공하지 않기때문에, scatter기반 회귀선과, 데이터 플롯을 직접 따로 그려야함


def analyze_sector_return_volatility(market=None, sector=None, month_ago=1):
    # 데이터 로드
    df = load_sector_data(market, month_ago=month_ago)  # 사용자 정의 데이터 로드 함수
    

    # 업종이 입력된 경우 해당 업종 내에서 필터링
    if sector is not None:
        if sector not in df['Sector'].unique():
            raise ValueError(f"'{sector}'는(은) 유효한 섹터가 아닙니다. 사용 가능한 섹터: {df['Sector'].unique()}")
        
        df = df[df['Sector'] == sector]
        
    else :
        # 해당 업종의 평균 수익률과 변동성 계산
        df = df.groupby('Sector').agg({
        'TotalReturn': 'mean',
        'Volatility': 'mean'
    }).reset_index().sort_values(by='TotalReturn', ascending=True)    
    
    if df.empty:
        raise ValueError("업종별 평균 수익률과 변동성 데이터가 없습니다.")
    
    # Plotly로 회귀선 데이터를 계산
    fig = px.scatter(
        df,
        x='TotalReturn',
        y='Volatility',
        trendline='ols'
    )
    
    # 회귀선 데이터를 추출
    
    # 이전에 생성한 fig 객체에서 회귀선의 결과를 가져옴 회귀선의 모델 결과를 포함하는 객체를 가져옵니다. 이 객체에는 회귀선의 절편(intercept)과 기울기(slope) 등의 정보가 포함되어 있음
    trendline_data = px.get_trendline_results(fig).iloc[0].px_fit_results
    
    # trendline_data.params: 회귀선의 파라미터(절편과 기울기)를 반환합니다. 여기서 intercept는 회귀선의 y절편을 의미하고, slope는 회귀선의 기울기를 나타냅니다.
    intercept, slope = trendline_data.params  

    # 회귀선 아래/위 판별 및 색상 추가
    df['Color'] = df.apply(
        # 현재 행의 변동성이 회귀선에서 예측한 변동성보다 낮은지를 확인, 회귀선의 방정식은 y=mx+b 형태이므로, 여기서 intercept는 y절편, slope는 기울기, row["TotalReturn"]는 x값
        lambda row: 'Below' if row['Volatility'] < (intercept + slope * row['TotalReturn']) \
                            else 'Above',axis=1
                            )
    
    # 회귀선 대비 아래에 있는 종목 수익률 내림차순으로 정렬한 df
    below_regression_line_df = df[df['Color'] == 'Below'].sort_values(by='TotalReturn', ascending=False)

    if sector is not None : 
        title = f'{market} 시장 {sector}업종 {month_ago}개월 수익률 대비 변동성 지수'
        trendline_name = f'회귀선({market} 시장 {sector}업종 {month_ago}개월 수익률 대비 평균 변동성)'
        hover_data = {'Name': True}
    else :
        title = f'{market} 시장 업종별 {month_ago}개월 수익률 대비 변동성 지수'
        trendline_name = f'회귀선({market} 시장 {month_ago}개월 수익률 대비 평균 변동성)'
        hover_data = {'Sector': True} 
        
        
    # 최종 Plotly 그래프 생성
    fig = px.scatter(
        df,
        x='TotalReturn',
        y='Volatility',
        color='Color',                  # 회귀선 기준 색상 설정
        hover_data=hover_data,
        title=title,
        labels={'TotalReturn': '평균 수익률', 'Volatility': '평균 변동성'},
        template="plotly_white"
    )

    # 회귀선 다시 추가
    fig.add_trace(go.Scatter(
        x=df['TotalReturn'],
        y=intercept + slope * df['TotalReturn'],
        mode="lines",
        name=trendline_name,
        line=dict(color='black', dash='dash')
    ))
    
    # 제목 가운데 정렬
    fig.update_layout(
        title_x=0.5,
        title_font=dict(size=20, color='black', family='Arial')
        ) 
    
    fig.show()
    
    # 변동성 대비 수익률이 좋은 종목 df 반환
    
    recommended_df = below_regression_line_df[below_regression_line_df['TotalReturn'] > 0]
    display(recommended_df)

In [14]:
analyze_sector_return_volatility('KOSPI','화학')

1개월 간의 data를 불러옵니다.


,Code,Name,Volatility,TotalReturn,Sector,Volume,Amount,Marcap,Risk,Color
791,005690,파미셀,3.97,48.92,화학,8302,74528630,536551658160,3,Below
138,178920,PI첨단소재,3.56,22.54,화학,354,6657360,552086853600,3,Below
630,014830,유니드,3.32,19.30,화학,304,22902200,507570000000,3,Below
215,007690,국도화학,2.27,18.83,화학,75,2424400,293295550800,2,Below
469,003720,삼영,3.19,13.60,화학,275,1055120,130220000000,3,Below
322,023450,동남합성,2.02,13.55,화학,122,4135800,118650000000,2,Below
657,005950,이수화학,2.85,9.73,화학,313,2029040,146788893940,2,Below
171,285130,SK케미칼,2.97,8.73,화학,388,17234200,766067965200,2,Below
376,004000,롯데정밀화학,2.66,8.34,화학,404,16160000,1032000000000,2,Below
827,004090,한국석유,2.12,5.93,화학,1061,14584250,174924973600,2,Below


In [13]:
analyze_sector_return_volatility('KOSPI')

1개월 간의 data를 불러옵니다.


,Sector,TotalReturn,Volatility,Color
56,조선,25.128182,3.268182,Below
17,디스플레이장비및부품,24.307500,3.440000,Below
23,반도체와반도체장비,22.250714,3.321429,Below
55,제약,10.008710,2.548387,Below
41,에너지장비및서비스,9.937500,2.818750,Below
5,건강관리장비와용품,9.486000,2.286000,Below
53,전자장비와기기,9.032593,2.723333,Below
57,종이와목재,8.146923,2.590000,Below
3,가정용기기와용품,7.340000,2.444000,Below
48,자동차부품,7.127808,2.575068,Below


In [12]:
analyze_sector_return_volatility('KOSDAQ')

1개월 간의 data를 불러옵니다.


,Sector,TotalReturn,Volatility,Color
60,통신장비,23.642553,4.330000,Below
23,반도체와반도체장비,23.070211,3.774155,Below
46,전기유틸리티,17.900000,3.760000,Below
14,기계,17.883472,4.179306,Below
65,핸드셋,17.762951,4.049016,Below
36,식품과기본식료품소매,16.880000,3.692000,Below
53,조선,15.086154,3.674615,Below
38,에너지장비및서비스,12.998000,3.716667,Below
19,디스플레이장비및부품,12.909846,3.238769,Below
50,전자장비와기기,12.410128,3.327821,Below


In [41]:
analyze_sector_return_volatility('ETF','채권')

3개월 간의 data를 불러옵니다.


,Code,Name,Volatility,TotalReturn,AvgReturn,Sector,Volume,Amount,Marcap,Color
222,461500,HANARO 종합채권(AA-이상)액티브,0.25,2.50,0.042936,채권,46002,5210,1151,Below
25,385540,RISE 종합채권(A-이상)액티브,0.20,2.42,0.042160,채권,57689,6253,13441,Below
54,451540,TIGER 종합채권(AA-이상)액티브,0.21,2.39,0.042173,채권,16333,912,6487,Below
96,454780,히어로즈 종합채권(AA-이상)액티브,0.23,2.32,0.038327,채권,199,21,3557,Below
47,356540,ACE 종합채권(AA-이상)KIS액티브,0.24,2.28,0.040919,채권,4419,471,7466,Below
43,436140,SOL 종합채권(AA-이상)액티브,0.21,2.27,0.039899,채권,37,4,8322,Below
117,451000,PLUS 종합채권(AA-이상)액티브,0.25,2.27,0.042024,채권,20,2,2658,Below
10,273130,KODEX 종합채권(AA-이상)액티브,0.23,2.26,0.041493,채권,62394,7283,28356,Below
438,272570,RISE 중장기국공채액티브,0.14,1.99,0.032178,채권,29,3,298,Below
131,438330,TIGER 투자등급회사채액티브,0.12,1.98,0.032577,채권,7558,869,2302,Below
